# 7.22 Compressible Fluids at Subsonic Velocity

Given that air is at a pressure of 19.3 psig and a temperature of 100 deg F and is flowing out of a 1/2" schedule 80 pipe into the atmosphere.
Find the flow rate of air in standard cubic feet per hour.

In [1]:
from fluids.units import *
from math import pi
dP = 19.3*u.psi
P = dP + 1*u.atm
P2 =  1*u.atm
T = 100*u.degF
L = 10*u.foot
NPS, D_pipe, Do_pipe, t = nearest_pipe(NPS=0.5, schedule=80)
A = 0.25*pi*D_pipe**2

fd = 0.0275 # assumed, initial guess
mu = 1.8e-8*u.Pa*u.s

In [2]:
# Take nitrogen-oxygen as air, and find the density and ratio
from thermo import PRMIX, Vm_to_rho
zs = [0.79, 0.21]
Tcs = [126.2, 154.58]
Pcs = [3394387.5, 5042945.25]
omegas = [0.04, 0.021]
MWs = [28.0134, 31.9988]
MW = sum(MWs[i]*zs[i] for i in range(2))

eos_flowing = PRMIX(T=(T).to(u.K).magnitude, P=P.to(u.Pa).magnitude, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
eos_std = PRMIX(T=288.15, P=101325.0, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
V_ratio = eos_flowing.V_g/eos_std.V_g
print('Ratio of actual to standard flow: %s' %(V_ratio))

rho = Vm_to_rho(eos_flowing.V_g, MW)*u.kg/u.m**3
print('Air density: %s' %(rho))

Ratio of actual to standard flow: 0.4663198131346627
Air density: 2.618303870512124 kilogram / meter ** 3


In [3]:
for i in range(5):
    # Problem says to consider 1 exit, and compressible friction
    K = K_from_f(fd=fd, L=L, D=D_pipe)
    K += exit_normal()
    # lump the two losses together for the `isothermal_gas` function
    fd_tot = f_from_K(L=L, D=D_pipe, K=K)
    m = isothermal_gas(rho=rho, fd=fd_tot, P1=P, P2=P2, L=L, D=D_pipe)
    Q = m/rho
    v = Q/A
    # update frictoin factor
    Re = Reynolds(D=D_pipe, rho=rho, mu=mu, V=v)
    fd = friction_factor(Re=Re, eD=0.0018*u.inch/D_pipe)
    Q_std = Q/V_ratio
print('Flow rate = %s' %(Q_std.to(u.ft**3/u.hour)))

Flow rate = 3773.572213074694 foot ** 3 / hour


The solution given in Crane is 3762 scfh. The solution there uses a simpler formula and does not iterate to converge the friction factor.